# A Python software engineer

Some insights. This is the beta verions:
- Sometimes the code interpreter cannot generate output files, it depends on the prompting.
- Sometimes, the source code generated have sintax errors.

In [14]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import os

# en windows es necesario especificar la ruta del .env
load_dotenv(Path("C:\projects\rag\.env"))
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

This is an example of a Python software engineer who can refactor, develop test units and run test units.

In [15]:
file = client.files.create(
  file=open("play_tetris.py", "rb"),
  purpose='assistants'
)

Some times, we cannot downlod the outputfile. We must ensure to include in instructinos, the name of the file and "Make the file id available to download".

In [16]:
output_file_name = "play_tetris_refactored.py"
assistant_instruction = r"""Generate {} file, always. Make the file id available to download. You are software engineer, your goal is to help another engineer write code using Python. 
  Focus on writting clean, performant and concise code to solve the problem.  """.format(output_file_name)

assistant = client.beta.assistants.create(
  name="Code writer assistant",
  description="Code writer assistant",  
  model="gpt-4o",
  instructions = assistant_instruction,
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }  
)

In [17]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Refactor this file.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [18]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [19]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_sLaFH2QFdkfAPM0w7DrfarB8', assistant_id='asst_RYoRdhXH2YX6gjTcRH1GHAJG', attachments=[Attachment(file_id='file-MTk3CpMQGyFXVckJ4bup5YKC', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=171, file_path=FilePath(file_id='file-MTk3CpMQGyFXVckJ4bup5YKC'), start_index=128, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')], value='The refactored Tetris script has been saved. You can download it using the link below:\n\n[Download the refactored Tetris script](sandbox:/mnt/data/play_tetris_refactored.py)\n\nFeel free to run and test it in your local environment where interactive input is supported.'), type='text')], created_at=1725937574, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_2cCmPSp7MW5WMZzBohDgIkBQ', status=None, thread_id='thread_glN3qEqmUo2i2UVuQd

Parsing results

In [20]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=171, file_path=FilePath(file_id='file-MTk3CpMQGyFXVckJ4bup5YKC'), start_index=128, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')], value='The refactored Tetris script has been saved. You can download it using the link below:\n\n[Download the refactored Tetris script](sandbox:/mnt/data/play_tetris_refactored.py)\n\nFeel free to run and test it in your local environment where interactive input is supported.')
[FilePathAnnotation(end_index=171, file_path=FilePath(file_id='file-MTk3CpMQGyFXVckJ4bup5YKC'), start_index=128, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')]
The refactored Tetris script has been saved. You can download it using the link below:

[Download the refactored Tetris script](sandbox:/mnt/data/play_tetris_refactored.py)

Feel free to run and test it in your local environment where interactive input is supported.


In [21]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

with open(output_file_name, "wb") as file:  
    file.write(file_name.content) 

# Unit tests

We have two option, we can procced in a similar way as the beginning to get the unit test; however, in this case we can continue in the same thread.

In [22]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Write a simple unit test for that file. Generate a play_tetris_unit_test.py file, always. Make the file id available to download.")

In [23]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [24]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_GRmqQsgKScbrngD1IlDn38ZV', assistant_id='asst_RYoRdhXH2YX6gjTcRH1GHAJG', attachments=[Attachment(file_id='file-08Grj9Yko9w7jut3Gcp79fWC', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=189, file_path=FilePath(file_id='file-08Grj9Yko9w7jut3Gcp79fWC'), start_index=147, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit test for the refactored Tetris script has been created. You can download it using the link below:\n\n[Download the Tetris unit test script](sandbox:/mnt/data/play_tetris_unit_test.py)\n\nFeel free to run and test it in your local environment to ensure the Tetris script works as expected.'), type='text')], created_at=1725937614, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_2lW7gFZliOnsyIoyWngjNAKZ', status=None, thread_id=

In [25]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=189, file_path=FilePath(file_id='file-08Grj9Yko9w7jut3Gcp79fWC'), start_index=147, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit test for the refactored Tetris script has been created. You can download it using the link below:\n\n[Download the Tetris unit test script](sandbox:/mnt/data/play_tetris_unit_test.py)\n\nFeel free to run and test it in your local environment to ensure the Tetris script works as expected.')
[FilePathAnnotation(end_index=189, file_path=FilePath(file_id='file-08Grj9Yko9w7jut3Gcp79fWC'), start_index=147, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')]
The unit test for the refactored Tetris script has been created. You can download it using the link below:

[Download the Tetris unit test script](sandbox:/mnt/data/play_tetris_unit_test.py)

Feel free to run and test it in your local environment to ensure the Tetris script works as expected.


In [26]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

with open("play_tetris_unit_test.py", "wb") as file:  
    file.write(file_name.content) 

# Asking for running the unit test

I'm not sure if code interpreter is runing the unit test, because the resuts of the assistant are different when I run the unit test.

In [27]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Run the unit test and show the results")

In [28]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [29]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_r6VUedoK3S6FNqiAYVqmfrzf', assistant_id='asst_RYoRdhXH2YX6gjTcRH1GHAJG', attachments=[Attachment(file_id='file-h8Vdh56vMBw2xWCW7CkZVym7', tools=[CodeInterpreterTool(type='code_interpreter')]), Attachment(file_id='file-8EzEAuoaGb5SC3mDnnNDaQa8', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=488, file_path=FilePath(file_id='file-8EzEAuoaGb5SC3mDnnNDaQa8'), start_index=445, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path'), FilePathAnnotation(end_index=572, file_path=FilePath(file_id='file-h8Vdh56vMBw2xWCW7CkZVym7'), start_index=530, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit tests have successfully passed after correcting the rotation logic. Here is the output:\n\n```\n....\n----------------------------------------------------------------------\nRan 4 tests in 0.001s\n\nOK\n

The results are different

In [30]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=488, file_path=FilePath(file_id='file-8EzEAuoaGb5SC3mDnnNDaQa8'), start_index=445, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path'), FilePathAnnotation(end_index=572, file_path=FilePath(file_id='file-h8Vdh56vMBw2xWCW7CkZVym7'), start_index=530, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit tests have successfully passed after correcting the rotation logic. Here is the output:\n\n```\n....\n----------------------------------------------------------------------\nRan 4 tests in 0.001s\n\nOK\n```\n\nThis indicates that all tests were executed successfully without any issues.\n\nYou can re-download the corrected refactored Tetris script and the unit test script using the links below:\n\n[Download the corrected refactored Tetris script](sandbox:/mnt/data/play_tetris_refactored.py)\n\n[Download the Tetris unit test script](sandbox:/mnt/data/play_tetris_unit_test.py)')
[FilePathAnnotati